In [ ]:
import pandas as pd
import numpy as np
import datalib_mt5 as dlm
import matplotlib.pyplot as plt
import matplotlib as mlt
import pickle
import time
import sqlite3
import tensorflow as tf
import thread_main_loop as tml
import thread_training
import queue
import threading
import struct
import itertools
import json
import random
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import RMSprop

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
print(tf.__version__)

In [ ]:
def y_columns(dataset, shift, fulltime):
    x_Mn = dataset
    x_Mn['t_DATE-TIME'] = x_Mn['t_DATE'] + x_Mn['t_TIME']
    x_Mn = x_Mn.drop_duplicates(subset=['t_DATE-TIME'])
    x_Mn = x_Mn.set_index(np.arange(len(x_Mn)))
    x_Mn['t_LAST_Shift'] = x_Mn['t_LAST'].shift(-shift)
    x_Mn['t_TIME_Shift'] = x_Mn['t_TIME'].shift(-shift)
    x_Mn['t_LAST_DELTA'] = x_Mn['t_LAST_Shift'] - x_Mn['t_LAST']
    x_Mn = x_Mn.dropna()

    for index, row in x_Mn.iterrows():
        if row['t_LAST_DELTA'] > target:
            x_Mn.loc[index,'t_POS_up'] = 1
            x_Mn.loc[index,'t_POS_down'] = 0
            x_Mn.loc[index,'t_POS_const'] = 0
        elif row['t_LAST_DELTA'] < -target:
            x_Mn.loc[index,'t_POS_down'] = 1
            x_Mn.loc[index,'t_POS_up'] = 0
            x_Mn.loc[index,'t_POS_const'] = 0
        else:
            x_Mn.loc[index,'t_POS_const'] = 1
            x_Mn.loc[index,'t_POS_up'] = 0
            x_Mn.loc[index,'t_POS_down'] = 0
    if !fulltime:
        x_Mn=x_Mn[x_Mn['t_TIME']<'14:00']
        x_Mn=x_Mn.set_index(np.arange(len(x_Mn)))
    return x_Mn

def process_dataset(dataset, is_categorical, shift, target, feature_list, x_matrix_size, fulltime)
    if is_categorical:
        y = tf.keras.utils.to_categorical(x_Mn[['t_POS_up','t_POS_down','t_POS_const']], num_classes=3)
    else:
        y = x_Mn['t_LAST_DELTA']
        
    x=x_Mn[feature_list]
    x_matrix = []
    y_matrix = y.loc[x_matrix_size:]
    y_matrix = pd.DataFrame(y_matrix)
    y_values_matrix = []

    
    for index, row in y_matrix.iterrows():
        x_matrix.append(x[index-x_matrix_size:index])
        y_values_matrix.append(row.values)
    x_values_matrix = []
    for element in x_matrix:
        x_values_matrix.append(element.values)
    x_values_matrix = np.array(x_values_matrix)
    y_values_matrix = np.array(y_values_matrix)
    
    x_processed=x_values_matrix
    y_processed=y_values_matrix
    
    return x_processed, y_processed
    
def class_lenght(y_processed):
    class_count=[]
    for i in range(len(y_processed[0])):
        class_count.append(len([y[i]>0 for y in y_processed]))
    print(class_count)
        
    
def under_sample(x_values_matrix, y_values_matrix):
    x_processed=[]
    y_processed=[]
    aux=0
    for i in range(len(y_values_matrix)):
        if np.array_equal(y_values_matrix[i],np.array([1,0,0])):
            x_processed.append(x_values_matrix[i])
            y_processed.append(y_values_matrix[i])
        if np.array_equal(y_values_matrix[i],np.array([0,1,0])):
            x_processed.append(x_values_matrix[i])
            y_processed.append(y_values_matrix[i])
        if np.array_equal(y_values_matrix[i],np.array([0,0,1])):
            if aux==8:
                x_processed.append(x_values_matrix[i])
                y_processed.append(y_values_matrix[i])
                aux=0
            else:
                aux+=1
                
#callbacks = myCallback()
class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.6):
            print("\nReached 60% accuracy so cancelling training!")
            self.model.stop_training = True
        
#callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
def scheduler(epoch, lr):
      if epoch < 10:
        return lr
      else:
        return lr * tf.math.exp(-0.1)

def variation_calc(a_matrix, Weight):
    line_lenght=(a_matrix.shape)[-2]-1
    variation_matrix=[]
    while line_lenght>0:
        i=0
        variation_line=[]
        for x in a_matrix[line_lenght]:
            var= (((abs(a_matrix[line_lenght][i])/abs(a_matrix[line_lenght-1][i]))-1)) * Weight
            variation_line.append(var)
            i+=1
        variation_matrix.append(variation_line)
        line_lenght-=1
    
    return variation_matrix

In [ ]:
dlm.insert_db()

In [ ]:
dataset = dlm.request_table()
dataset.t_DATE.unique()

In [ ]:
#full_feature_list = ['t_BID','t_ASK','t_LAST','iAC','iAD','iADX','iADXWilder','iAlligator','iAMA','iAO','iATR','iBearsPower','iBands','iBullsPower','iCCI','iChaikin','iDEMA','iDeMarker','iEnvelopes','iForce','iFractals','iFrAMA','iIchimoku','iBWMFI','iMomentum','iMFI','iMA','iOsMA','iMACD','iOBV','iSAR','iRSI','iRVI','iStdDEV','iStochastic','iTEMA','iTriX','iWPR','iVIDyA','iVolumes']
shift = 10
target = 10
feature_list = ['iBands','iBullsPower','iCCI','iChaikin','iDEMA','iDeMarker','iEnvelopes','iForce','iFrAMA']
tim_step = 60

In [ ]:
dataset=y_columns(dataset, shift, True)

## Train test split

In [ ]:
split =int(len(x_processed)*0.7)
print(split)

In [ ]:
#train_X, test_X, train_y, test_y = train_test_split(x_processed, y_processed, random_state = 0)
train_X, test_X, train_y, test_y = x_processed[:split], x_processed[split:], y_processed[:split], y_processed[split:]
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
train_y=np.squeeze(train_y,axis=-1)
test_y=np.squeeze(test_y,axis=-1)
scaler = MinMaxScaler()
test_y_scaled = scaler.fit_transform(test_y.reshape(-1,1))
train_y_scaled = scaler.fit_transform(train_y.reshape(-1,1))
test_y_scaled = np.squeeze(test_y_scaled,axis=-1)
train_y_scaled = np.squeeze(train_y_scaled,axis=-1)

## Over sampling

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE('minority')

In [ ]:
train_X_squeeze=train_X.reshape(len(train_X),len(train_X[0]) * len(train_X[0][0]))
print(train_X_squeeze.shape)

In [ ]:
#layer = tf.keras.layers.LayerNormalization(axis=1)
#output = layer(train_X[rand])

In [ ]:
train_X_os, train_y_os = smote.fit_sample(train_X_squeeze, train_y)
print(train_X_os.shape, train_y_os.shape)

In [ ]:
train_X_os, train_y_os = smote.fit_sample(train_X_os, train_y_os)

In [ ]:
print(train_X_os.shape, train_y_os.shape)

In [ ]:
up=0
down=0
const=0
for i in range(len(train_y_os)):
    if np.array_equal(train_y_os[i],np.array([1,0,0])):
        up+=1
    if np.array_equal(train_y_os[i],np.array([0,1,0])):
        down+=1
    if np.array_equal(train_y_os[i],np.array([0,0,1])):
        const+=1
print(up)
print(down)
print(const)

In [ ]:
train_X_os = train_X_os.reshape(len(train_X_os), len(train_X[0]), len(train_X[0][0]))
print(train_X_os.shape, train_y_os.shape)

## Visualize

In [ ]:
ready = False

In [ ]:
#plt.imshow(train_X[index], norm=norm)
#norm = mlt.colors.Normalize(vmin=0.,vmax=1.)
layer = tf.keras.layers.LayerNormalization(axis=1)
fig=plt.figure(figsize=(18,30))
for i in range(5):
    while (not ready):
        rand = random.randint(0, len(train_y)-1)
        if np.array_equal(train_y[rand],[1.0, 0., 0.]):
            plt.subplot(1,5,i+1)
            plt.subplot(1,5,i+1)
            
            output = layer(train_X[rand])
            plt.imshow(output)
            #plt.imshow(train_X[rand], norm=norm)
            plt.title(str(rand)+' / '+str(train_y[rand]))
            ready = True
    ready=False

In [ ]:
#plt.imshow(train_X[index], norm=norm)
#norm = mlt.colors.Normalize(vmin=0.,vmax=1.)
layer = tf.keras.layers.LayerNormalization(axis=1)
fig=plt.figure(figsize=(18,30))
for i in range(5):
    while (not ready):
        rand = random.randint(0, len(train_y)-1)
        if np.array_equal(train_y[rand],[0, 1.0, 0.]):
            plt.subplot(1,5,i+1)
            plt.subplot(1,5,i+1)
            
            output = layer(train_X[rand])
            plt.imshow(output)
            #plt.imshow(train_X[rand], norm=norm)
            plt.title(str(rand)+' / '+str(train_y[rand]))
            ready = True
    ready=False

In [ ]:
#plt.imshow(train_X[index], norm=norm)
#norm = mlt.colors.Normalize(vmin=0.,vmax=1.)
layer = tf.keras.layers.LayerNormalization(axis=1)
fig=plt.figure(figsize=(18,30))
for i in range(5):
    while (not ready):
        rand = random.randint(0, len(train_y)-1)
        if np.array_equal(train_y[rand],[0., 0., 1.]):
            plt.subplot(1,5,i+1)
            plt.subplot(1,5,i+1)
            
            output = layer(train_X[rand])
            plt.imshow(output)
            #plt.imshow(train_X[rand], norm=norm)
            plt.title(str(rand)+' / '+str(train_y[rand]))
            ready = True
    ready=False

In [ ]:
plt.close('all')

In [ ]:
i=0
layer = tf.keras.layers.LayerNormalization(axis=1)
for x in train_X:
    fig=plt.figure(figsize=(18,30))
    output = layer(x)
    plt.imshow(output)
    plt.title(str(i))
    if (train_y[i]==1.0):
        fig.savefig('train/up/'+str(i)+'.png')
    elif (train_y[i]==-1.0):
        fig.savefig('train/down/'+str(i)+'.png')
    elif (train_y[i]==0):
        fig.savefig('train/const/'+str(i)+'.png')
    plt.close(fig)
    i+=1
plt.close('all')

In [ ]:
i=0
for x in test_X:
    fig=plt.figure(figsize=(18,30))
    output = layer(x)
    plt.imshow(output)
    plt.title(str(i))
    if (test_y[i]==1.0):
        fig.savefig('test/up/'+str(i)+'.png')
    elif (test_y[i]==-1.0):
        fig.savefig('test/down/'+str(i)+'.png')
    elif (test_y[i]==0):
        fig.savefig('test/const/'+str(i)+'.png')
    plt.close(fig)
    i+=1
plt.close('all')

In [ ]:
layer3=tf.keras.layers.LayerNormalization()
output = layer3(train_X[837])
plt.imshow(output)

## =================================================================

In [ ]:
#callbacks = myCallback()
class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.6):
            print("\nReached 60% accuracy so cancelling training!")
            self.model.stop_training = True
        
#callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
def scheduler(epoch, lr):
      if epoch < 10:
        return lr
      else:
        return lr * tf.math.exp(-0.1)

In [ ]:
def variation_calc(a_matrix, Weight):
    line_lenght=(a_matrix.shape)[-2]-1
    variation_matrix=[]
    while line_lenght>0:
        i=0
        variation_line=[]
        for x in a_matrix[line_lenght]:
            var= (((abs(a_matrix[line_lenght][i])/abs(a_matrix[line_lenght-1][i]))-1)) * Weight
            variation_line.append(var)
            i+=1
        variation_matrix.append(variation_line)
        line_lenght-=1
    
    variation_matrix = a_matrix[1:]+(a_matrix[1:]*(variation_matrix))
    return variation_matrix

In [ ]:
#tf.keras.layers.Activation('relu')
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(train_X[0].shape)))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),input_shape=[None]))
model.add(tf.keras.layers.SimpleRNN(100, return_sequences=True))
model.add(tf.keras.layers.Dropout(0.6))
model.add(tf.keras.layers.SimpleRNN(100))
model.add(tf.keras.layers.Dropout(0.8))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(100,kernel_regularizer=tf.keras.regularizers.L1(0.001),activity_regularizer=tf.keras.regularizers.L2(0.01)))


model.add(tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),input_shape=[None]))
model.add(tf.keras.layers.SimpleRNN(50, return_sequences=True))
model.add(tf.keras.layers.Dropout(0.6))
model.add(tf.keras.layers.SimpleRNN(50))
model.add(tf.keras.layers.Dropout(0.8))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(32,kernel_regularizer=tf.keras.regularizers.L1(0.001),activity_regularizer=tf.keras.regularizers.L2(0.01)))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

optimizer = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=["accuracy"])
model.summary()

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LayerNormalization(input_shape=(train_X[0].shape)))


#model.add(tf.keras.layers.Conv1D(64, 5, activation='relu'))
#model.add(tf.keras.layers.MaxPooling1D(pool_size=4))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),input_shape=[None]))
model.add(tf.keras.layers.LSTM(100, return_sequences=True))
model.add(tf.keras.layers.LSTM(32))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64,kernel_regularizer=tf.keras.regularizers.L1(0.001),activity_regularizer=tf.keras.regularizers.L2(0.01)))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(3, activation= 'softmax'))

#model.add(tf.keras.layers.Lambda(lambda x: tf.squeeze(x, axis=-1),input_shape=[None]))
#model.add(tf.keras.layers.SimpleRNN(100))
#model.add(tf.keras.layers.Dense(1, activation= 'relu'))
#optimizer = 'adam'
optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=[tf.keras.metrics.CategoricalCrossentropy(),"accuracy"])
model.summary()

In [ ]:
callback1 = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit(train_X, train_y, epochs=100,batch_size=32, validation_data=(test_X, test_y),verbose=2,shuffle=False)

In [ ]:
# Over-sampling model loss: 0.3165 - categorical_crossentropy: 0.2438 - accuracy: 0.9091 - val_loss: 0.7229 - val_categorical_crossentropy: 0.6535 - val_accuracy: 0.7582
model.save('models/LSTMM10609-class-015')

In [ ]:
plt.plot(history.history['accuracy'][:], label='train')
plt.plot(history.history['val_accuracy'][:], label='test')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'][:], label='train')
plt.plot(history.history['val_loss'][:], label='test')
plt.legend()
plt.show()

In [ ]:
t1 = thread_training.thread_training(model, train_X, train_y, 1000, 72, (test_X, test_y), 1, False) 
t1_history = t1.start()

In [ ]:
t2 = thread_training.thread_training(model, train_X, train_y, 1000, 72, (test_X, test_y), 1, False) 
t2_history = t2.start()

In [ ]:
loaded_model = tf.keras.models.load_model('models/LSTH1609-class-007')
loaded_model.summary()

In [ ]:
model = loaded_model

In [ ]:
temp_x = train_X_os
temp_y = train_y_os

In [ ]:
x_evaluate = x_processed
y_evaluate = y_processed

In [ ]:
model.evaluate(x_evaluate, y_evaluate, batch_size=144)

In [ ]:
yhat = model.predict(test_X)

In [ ]:
ps = yhat[:100]
ts = test_y[:100]

In [ ]:
plt.figure(figsize=(20, 6))
#dlm.plot_series(np.arange(len(ps)), ps[:])
plt.subplot(211)
plt.plot(np.arange(len(ps)), [x[0] for x in ps[:]])
#plt.subplot(212)
plt.plot(np.arange(len(ts)), [x[0] for x in ts[:]])
plt.grid()

In [ ]:
plt.figure(figsize=(20, 6))
#dlm.plot_series(np.arange(len(ps)), ps[:])
plt.subplot(211)
plt.plot(np.arange(len(ps)),ps)
#plt.subplot(212)
plt.plot(np.arange(len(ts)),ts)
plt.grid(True)

## =================================================================

In [ ]:
import os
os.system('shutdown -s')

In [ ]:
t1 = tml.thread_main_loop(model, 60, 9, 0, "0", "1",1)
t1.start()

In [ ]:
t1.kill()
t1.join()

In [ ]:
model = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Lambda(lambda x: tf.reshape(np.array(x), [None, 16,16,2,1]),input_shape=[None]),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(104, activation='softmax')
    ])